In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-i8bexrlr
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-i8bexrlr
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=d875a02655d317565e4082b31cc789edc874098b8ec68e39d36f19be54b7c64f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4i2apq0m/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
!nvidia-smi

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
    int main()
{
    std::cout << "Welcome To GeeksforGeeks\n";
    return 0;
}

Welcome To GeeksforGeeks



In [ ]:
%%cu
#include <iostream>
__global__ void kernel( void ) {
}
int main( void ) {
kernel<<<1,1>>>();
printf( "Hello, World!\n" );
return 0;
}

Hello, World!



In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>

using namespace std;
int main() {
    srand(time(0)); //numeros aleatorios

    const int size = 1024;
    unsigned char arr[size]; // arreglo original
    unsigned char complemento[size]; // arreglo de complementos

    // llena el arreglo original con valores aleatorios y calcular sus complementos
    for(int i = 0; i < size; i++) {
        arr[i] = rand() % 256; // valor aleatorio entre 0 y 255
        complemento[i] = 255 - arr[i]; // complemento a un byte
    }

    // imprimir los valores originales y sus complementos
    for(int i = 0; i < size; i++) {
        cout << "Valor original: " << (int)arr[i] << " -> Complemento: " << (int)complemento[i] << endl;
    }

    return 0;
}




Valor original: 78 -> Complemento: 177
Valor original: 75 -> Complemento: 180
Valor original: 198 -> Complemento: 57
Valor original: 187 -> Complemento: 68
Valor original: 146 -> Complemento: 109
Valor original: 249 -> Complemento: 6
Valor original: 153 -> Complemento: 102
Valor original: 41 -> Complemento: 214
Valor original: 211 -> Complemento: 44
Valor original: 80 -> Complemento: 175
Valor original: 55 -> Complemento: 200
Valor original: 143 -> Complemento: 112
Valor original: 244 -> Complemento: 11
Valor original: 73 -> Complemento: 182
Valor original: 105 -> Complemento: 150
Valor original: 55 -> Complemento: 200
Valor original: 85 -> Complemento: 170
Valor original: 112 -> Complemento: 143
Valor original: 249 -> Complemento: 6
Valor original: 98 -> Complemento: 157
Valor original: 137 -> Complemento: 118
Valor original: 57 -> Complemento: 198
Valor original: 207 -> Complemento: 48
Valor original: 185 -> Complemento: 70
Valor original: 145 -> Complemento: 110
Valor original: 247 

int idx: calcula el índice único para cada hilo. En CUDA, los hilos se organizan en bloques, y los bloques se organizan en una cuadrícula. threadIdx.x es el índice del hilo dentro de su bloque, blockIdx.x es el índice del bloque dentro de la cuadrícula, y blockDim.x es el número de hilos en cada bloque.

"cudaDeviceSynchronize();: Esta línea hace que el codigo en la CPU espere hasta que todos los hilos en la GPU hayan terminado. Esto es necesario porque el código en la CPU puede continuar ejecutándose mientras el kernel se está ejecutando en la GPU, y queremos asegurarnos de que el kernel haya terminado antes de continuar."

In [ ]:
%%cu
#include <iostream>
#include <curand.h>
#include <curand_kernel.h>

using namespace std;

__global__ void llenarArrays(unsigned char* arr, unsigned char* complemento, int size) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < size) {
        curandState state;
        curand_init(clock64(), idx, 0, &state);
        arr[idx] = curand_uniform(&state) * 256; // valor aleatorio entre 0 y 255
        complemento[idx] = 255 - arr[idx]; // complemento a un byte
    }
}

int main() {
    const int size = 1024;
    unsigned char *arr, *complemento;

    // Reserva de memoria en el dispositivo
    cudaMallocManaged(&arr, size * sizeof(unsigned char));
    cudaMallocManaged(&complemento, size * sizeof(unsigned char));

    // Lanzamiento del kernel
    llenarArrays<<<size / 256 + 1, 256>>>(arr, complemento, size);

    // Espera a que todos los hilos hayan terminado
    cudaDeviceSynchronize();

    // Imprimir los valores originales y sus complementos
    for(int i = 0; i < size; i++) {
        cout << "Valor original: " << (int)arr[i] << " -> Complemento: " << (int)complemento[i] << endl;
    }

    // Liberación de memoria
    cudaFree(arr);
    cudaFree(complemento);

    return 0;
}


Valor original: 97 -> Complemento: 158
Valor original: 38 -> Complemento: 217
Valor original: 157 -> Complemento: 98
Valor original: 36 -> Complemento: 219
Valor original: 42 -> Complemento: 213
Valor original: 110 -> Complemento: 145
Valor original: 164 -> Complemento: 91
Valor original: 224 -> Complemento: 31
Valor original: 166 -> Complemento: 89
Valor original: 62 -> Complemento: 193
Valor original: 148 -> Complemento: 107
Valor original: 160 -> Complemento: 95
Valor original: 22 -> Complemento: 233
Valor original: 241 -> Complemento: 14
Valor original: 147 -> Complemento: 108
Valor original: 77 -> Complemento: 178
Valor original: 99 -> Complemento: 156
Valor original: 221 -> Complemento: 34
Valor original: 229 -> Complemento: 26
Valor original: 245 -> Complemento: 10
Valor original: 74 -> Complemento: 181
Valor original: 50 -> Complemento: 205
Valor original: 255 -> Complemento: 0
Valor original: 53 -> Complemento: 202
Valor original: 205 -> Complemento: 50
Valor original: 104 -> 

In [ ]:
%%cu
#include <iostream>
#include <curand.h>
#include <curand_kernel.h>

using namespace std;

__global__ void llenarArrays(unsigned char* arr, unsigned char* complemento, int size) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < size) {
        curandState state;
        curand_init(clock64(), idx, 0, &state);
        arr[idx] = curand_uniform(&state) * 256; // valor aleatorio entre 0 y 255
        complemento[idx] = 255 - arr[idx]; // complemento a un byte
    }
}

__global__ void imprimirValores(unsigned char* arr, unsigned char* complemento, int size) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < size) {
        printf("Hilo: %d -> Valor original: %d, Complemento: %d\n", idx, (int)arr[idx], (int)complemento[idx]);
    }
}

int main() {
    const int size = 1024;
    unsigned char *arr, *complemento;

    // Reserva de memoria en el dispositivo
    cudaMallocManaged(&arr, size * sizeof(unsigned char));
    cudaMallocManaged(&complemento, size * sizeof(unsigned char));

    // Lanzamiento del kernel
    llenarArrays<<<size / 256 + 1, 256>>>(arr, complemento, size);

    // Espera a que todos los hilos hayan terminado
    cudaDeviceSynchronize();

    // Lanzamiento del kernel para imprimir los valores
    imprimirValores<<<size / 256 + 1, 256>>>(arr, complemento, size);

    // Espera a que todos los hilos hayan terminado
    cudaDeviceSynchronize();

    // Liberación de memoria
    cudaFree(arr);
    cudaFree(complemento);

    return 0;
}


Hilo: 352 -> Valor original: 160, Complemento: 95
Hilo: 353 -> Valor original: 234, Complemento: 21
Hilo: 354 -> Valor original: 192, Complemento: 63
Hilo: 355 -> Valor original: 191, Complemento: 64
Hilo: 356 -> Valor original: 13, Complemento: 242
Hilo: 357 -> Valor original: 217, Complemento: 38
Hilo: 358 -> Valor original: 126, Complemento: 129
Hilo: 359 -> Valor original: 170, Complemento: 85
Hilo: 360 -> Valor original: 147, Complemento: 108
Hilo: 361 -> Valor original: 114, Complemento: 141
Hilo: 362 -> Valor original: 148, Complemento: 107
Hilo: 363 -> Valor original: 173, Complemento: 82
Hilo: 364 -> Valor original: 102, Complemento: 153
Hilo: 365 -> Valor original: 66, Complemento: 189
Hilo: 366 -> Valor original: 24, Complemento: 231
Hilo: 367 -> Valor original: 14, Complemento: 241
Hilo: 368 -> Valor original: 95, Complemento: 160
Hilo: 369 -> Valor original: 90, Complemento: 165
Hilo: 370 -> Valor original: 122, Complemento: 133
Hilo: 371 -> Valor original: 60, Complemento

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>

int main() {
    srand(time(0)); // Para generar números aleatorios

    int numFilas = 10; // Ajusta esto a la cantidad de hilos por bloque
    int numColumnas = 10; // Ajusta esto a la cantidad de hilos por bloque

    // Crear las dos matrices
    int** matriz1 = new int*[numFilas];
    int** matriz2 = new int*[numFilas];
    for(int i = 0; i < numFilas; ++i) {
        matriz1[i] = new int[numColumnas];
        matriz2[i] = new int[numColumnas];
    }

    // Llenar las matrices con valores aleatorios en el rango de un byte
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            matriz1[i][j] = rand() % 256;
            matriz2[i][j] = rand() % 256;
        }
    }

    // Imprimir las matrices (opcional)
    std::cout << "Matriz 1:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matriz1[i][j] << " ";
        }
        std::cout << std::endl;
    }

    std::cout << "Matriz 2:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matriz2[i][j] << " ";
        }
        std::cout << std::endl;
    }

    // Liberar la memoria
    for(int i = 0; i < numFilas; ++i) {
        delete [] matriz1[i];
        delete [] matriz2[i];
    }
    delete [] matriz1;
    delete [] matriz2;

    return 0;
}


Matriz 1:
100 7 124 72 18 196 67 78 134 134 
227 14 85 152 16 73 5 236 11 177 
210 246 136 93 88 176 102 234 182 162 
113 59 60 248 92 101 30 67 161 181 
131 53 35 165 161 160 50 137 3 183 
42 141 120 239 112 19 10 220 103 8 
200 26 132 106 36 230 61 151 67 207 
43 166 42 100 4 153 20 199 87 31 
82 202 120 70 232 110 248 73 151 180 
25 103 1 198 96 213 235 67 1 31 
Matriz 2:
21 120 186 47 157 217 19 72 19 161 
131 174 248 230 239 116 81 129 52 29 
117 21 69 14 227 59 190 187 130 198 
235 118 39 72 169 46 92 167 160 249 
101 233 31 218 71 18 219 111 129 95 
28 72 208 25 165 243 72 46 129 8 
168 251 163 135 33 78 115 182 134 179 
226 54 130 145 108 204 180 152 49 124 
5 144 97 187 21 19 20 34 173 155 
77 46 47 89 230 179 159 100 137 233 



In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>

using namespace std;
int main() {
    srand(time(0)); // generar números aleatorios

    int numFilas = 10; //
    int numColumnas = 10; //

    //  matrices
    int** matriz1 = new int*[numFilas];
    int** matriz2 = new int*[numFilas];
    int** matrizProducto = new int*[numFilas];
    for(int i = 0; i < numFilas; ++i) {
        matriz1[i] = new int[numColumnas];
        matriz2[i] = new int[numColumnas];
        matrizProducto[i] = new int[numColumnas];
    }

    // Llenar las primeras dos matrices con valores aleatorios en el rango de un byte
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            matriz1[i][j] = rand() % 256;
            matriz2[i][j] = rand() % 256;
        }
    }

    // Calcular el producto de las dos matrices
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            matrizProducto[i][j] = 0;
            for(int k = 0; k < numColumnas; ++k) {
                matrizProducto[i][j] += matriz1[i][k] * matriz2[k][j];
            }
        }
    }

    // Imprimir las tres matrices
    std::cout << "Matriz 1:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            cout << matriz1[i][j] << " ";
        }
        cout << endl;
    }

    std::cout << "Matriz 2:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            cout << matriz2[i][j] << " ";
        }
        cout << endl;
    }

    cout << "Matriz Producto:" << endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matrizProducto[i][j] << " ";
        }
        cout << endl;
    }

    // Liberar la memoria
    for(int i = 0; i < numFilas; ++i) {
        delete [] matriz1[i];
        delete [] matriz2[i];
        delete [] matrizProducto[i];
    }
    delete [] matriz1;
    delete [] matriz2;
    delete [] matrizProducto;

    return 0;
}


Matriz 1:
5 146 116 213 14 74 29 180 117 182 
153 107 64 90 173 102 177 199 99 179 
155 65 245 166 171 142 33 7 99 168 
73 195 6 32 38 220 210 121 18 49 
124 1 184 254 106 44 180 173 121 37 
162 96 30 59 249 237 43 19 246 255 
184 19 27 39 143 52 77 165 110 155 
76 221 118 90 1 205 134 38 162 54 
154 98 0 83 231 214 119 170 163 90 
94 94 9 134 209 220 39 83 69 150 
Matriz 2:
202 20 107 141 255 207 247 238 182 36 
196 195 106 71 231 178 248 38 156 114 
254 184 245 106 93 68 249 97 97 16 
14 250 138 106 212 193 29 199 31 189 
191 157 8 28 166 179 239 187 206 159 
1 116 217 48 108 117 238 227 17 96 
44 108 193 149 76 49 149 107 224 103 
136 119 240 109 90 185 225 161 77 49 
107 231 7 110 238 51 179 237 4 164 
40 228 132 171 188 107 62 39 166 44 
Matriz Producto:
110375 207021 163635 117822 181727 147346 170510 146703 96719 107270 
155144 203672 185573 144797 217074 185030 248463 201525 175989 125133 
161124 211244 169508 126938 214523 165513 229518 195043 141116 116475 
91998 132484 15857

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <curand.h>
#include <curand_kernel.h>

__global__ void createMatrices(int** matriz1, int** matriz2, int numFilas, int numColumnas) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < numFilas && j < numColumnas) {
        curandGenerator_t gen;
        curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
        curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
        curandGenerateUniform(gen, matriz1[i] + j, 1);
        curandGenerateUniform(gen, matriz2[i] + j, 1);
        curandDestroyGenerator(gen);
    }
}

__global__ void calculateProduct(int** matriz1, int** matriz2, int** matrizProducto, int numFilas, int numColumnas) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < numFilas && j < numColumnas) {
        matrizProducto[i][j] = 0;
        for(int k = 0; k < numColumnas; ++k) {
            matrizProducto[i][j] += matriz1[i][k] * matriz2[k][j];
        }
    }
}

int main() {
    srand(time(0));
    int numFilas = 10;
    int numColumnas = 10;
    int** matriz1 = new int*[numFilas];
    int** matriz2 = new int*[numFilas];
    int** matrizProducto = new int*[numFilas];
    for(int i = 0; i < numFilas; ++i) {
        matriz1[i] = new int[numColumnas];
        matriz2[i] = new int[numColumnas];
        matrizProducto[i] = new int[numColumnas];
    }

    // Crear matrices
    dim3 threadsPerBlock(16, 16);
    dim3 numBlocks((numFilas + threadsPerBlock.x - 1) / threadsPerBlock.x, (numColumnas + threadsPerBlock.y - 1) / threadsPerBlock.y);
    createMatrices<<<numBlocks, threadsPerBlock>>>(matriz1, matriz2, numFilas, numColumnas);

    // Calcular el producto de las dos matrices
    calculateProduct<<<numBlocks, threadsPerBlock>>>(matriz1, matriz2, matrizProducto, numFilas, numColumnas);

    std::cout << "Matriz 1:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matriz1[i][j] << " ";
        }
        std::cout << std::endl;
    }
    std::cout << "Matriz 2:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matriz2[i][j] << " ";
        }
        std::cout << std::endl;
    }
    std::cout << "Matriz Producto:" << std::endl;
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            std::cout << matrizProducto[i][j] << " ";
        }
        std::cout << std::endl;
    }
    for(int i = 0; i < numFilas; ++i) {
        delete [] matriz1[i];
        delete [] matriz2[i];
        delete [] matrizProducto[i];
    }
    delete [] matriz1;
    delete [] matriz2;
    delete [] matrizProducto;
    return 0;
}



/tmp/tmpij9c7llg/a5dac5ee-4fed-43aa-8217-094538a36da1.cu(14): error: argument of type "int *" is incompatible with parameter of type "float *"

/tmp/tmpij9c7llg/a5dac5ee-4fed-43aa-8217-094538a36da1.cu(15): error: argument of type "int *" is incompatible with parameter of type "float *"

2 errors detected in the compilation of "/tmp/tmpij9c7llg/a5dac5ee-4fed-43aa-8217-094538a36da1.cu".



In [ ]:
import numpy as np

# Matriz A
A = np.array([
    [229, 236, 114, 160, 27, 77, 188, 225, 175, 173],
    [249, 90, 190, 117, 138, 222, 253, 252, 50, 29],
    [74, 57, 43, 10, 205, 255, 223, 122, 120, 171],
    [200, 27, 140, 18, 164, 165, 217, 25, 63, 42],
    [97, 9, 182, 169, 240, 51, 236, 21, 87, 130],
    [161, 152, 237, 28, 233, 193, 37, 165, 85, 185],
    [200, 16, 1, 26, 168, 60, 232, 161, 241, 135],
    [211, 129, 14, 27, 137, 212, 246, 255, 215, 141],
    [199, 36, 217, 119, 44, 112, 75, 72, 39, 88],
    [1, 136, 71, 4, 152, 174, 216, 153, 82, 122]
])

# Matriz B
B = np.array([
    [169, 148, 126, 50, 66, 211, 190, 130, 62, 149],
    [22, 146, 169, 89, 226, 49, 207, 123, 105, 70],
    [132, 154, 143, 150, 127, 117, 167, 189, 144, 38],
    [35, 56, 237, 32, 193, 0, 208, 236, 124, 45],
    [142, 73, 163, 142, 26, 26, 93, 97, 231, 20],
    [134, 217, 174, 225, 79, 130, 27, 57, 101, 65],
    [212, 101, 120, 0, 81, 137, 127, 203, 173, 106],
    [124, 77, 167, 86, 192, 165, 155, 23, 209, 1],
    [60, 111, 111, 163, 225, 134, 184, 27, 189, 111],
    [118, 177, 138, 164, 105, 173, 83, 110, 96, 91]
])

# Producto de las matrices
producto = np.dot(A, B)
print(producto)


[[177363 199903 244193 152080 236561 200193 254580 192206 218480 123039]
 [213886 199692 244601 156828 187783 199585 223632 194697 229727 108335]
 [172848 172260 189313 160124 137074 158262 144574 131663 194288  90416]
 [156742 142156 152495 113699  98817 135179 136932 136289 150881  85667]
 [160640 139869 187240 121221 130809 125017 169561 179226 188854  84243]
 [176999 201598 225022 192920 175013 175985 195606 158580 211569  92457]
 [166628 145305 172141 125031 149173 169267 174099 126165 194610 103177]
 [202462 203038 231435 172248 202445 212582 215242 154806 235074 121763]
 [126040 135330 154549 107941 119702 127451 148534 137661 127440  74291]
 [141653 144309 166672 130198 136421 128685 136226 118884 174303  70150]]


In [ ]:
%%cu
#include <iostream>


#include <curand.h>
#include <curand_kernel.h>

#define BLOCK_SIZE 16

using namespace std;
__global__ void iniMatriz(int* matriz, int numFilas, int numColumnas) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < numFilas && col < numColumnas) {
        curandState state;
        curand_init(clock64(), row * numColumnas + col, 0, &state);
        matriz[row * numColumnas + col] = curand(&state) % 256;
    }
}

__global__ void sacarProducto(int* matriz1, int* matriz2, int* sacarProducto, int numFilas, int numColumnas) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if(row < numFilas && col < numColumnas) {
        int sum = 0;
        for(int i = 0; i < numColumnas; ++i) {
            sum += matriz1[row * numColumnas + i] * matriz2[i * numColumnas + col];
        }
        sacarProducto[row * numColumnas + col] = sum;
    }
}

void printMatriz(int* matriz, int numFilas, int numColumnas) {
    for(int i = 0; i < numFilas; ++i) {
        for(int j = 0; j < numColumnas; ++j) {
            cout << matriz[i * numColumnas + j] << " ";
        }
        cout << endl;
    }
}

int main() {
    int numFilas = 3;
    int numColumnas = 3;

    int* matriz1;
    int* matriz2;
    int* matrizProducto;

    cudaMallocManaged(&matriz1, numFilas * numColumnas * sizeof(int));
    cudaMallocManaged(&matriz2, numFilas * numColumnas * sizeof(int));
    cudaMallocManaged(&matrizProducto, numFilas * numColumnas * sizeof(int));

    dim3 hilosPorBlock(BLOCK_SIZE, BLOCK_SIZE);
    dim3 numBlocks((numColumnas + hilosPorBlock.x - 1) / hilosPorBlock.x, (numFilas + hilosPorBlock.y - 1) / hilosPorBlock.y);

    iniMatriz<<<numBlocks, hilosPorBlock>>>(matriz1, numFilas, numColumnas);
    cudaDeviceSynchronize();

    iniMatriz<<<numBlocks, hilosPorBlock>>>(matriz2, numFilas, numColumnas);
    cudaDeviceSynchronize();

    sacarProducto<<<numBlocks, hilosPorBlock>>>(matriz1, matriz2, matrizProducto, numFilas, numColumnas);
    cudaDeviceSynchronize();

    cout << "Matriz 1:" << endl;
    printMatriz(matriz1, numFilas, numColumnas);

    cout << "Matriz 2:" << endl;
    printMatriz(matriz2, numFilas, numColumnas);

    cout << "Matriz Producto:" << endl;
    printMatriz(matrizProducto, numFilas, numColumnas);

    cudaFree(matriz1);
    cudaFree(matriz2);
    cudaFree(matrizProducto);

    return 0;
}


Matriz 1:
227 192 143 
252 144 150 
89 63 26 
Matriz 2:
177 194 118 
60 99 192 
172 131 182 
Matriz Producto:
76295 81779 89676 
79044 82794 84684 
24005 26909 27330 



In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>

#define SIZE 1024

using namespace std;
void bubbleSort(int arr[], int n) {
    for(int i = 0; i < n-1; i++) {
        for (int j = 0; j < n-i-1; j++) {
            if (arr[j] > arr[j+1]) {
                swap(arr[j], arr[j+1]);
            }
        }
    }
}

void imprimirArr(int arr[], int n) {
    for(int i = 0; i < n; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
}

int main() {
    srand(time(0));
    int arr[SIZE];
    for(int i = 0; i < SIZE; i++) {
        arr[i] = rand() % 10001;
    }

    cout << "Arreglo desordenado: " << endl;
    imprimirArr(arr, SIZE);

    bubbleSort(arr, SIZE);

    cout << "Arreglo ordenado: " << endl;
    imprimirArr(arr, SIZE);

    return 0;
}



Arreglo desordenado: 
5992 6304 1195 3103 6189 2990 3912 1397 2613 8759 9827 412 5983 4188 5452 7077 4767 6686 7910 6 4892 6928 5831 8080 8375 5318 2919 5418 5883 2406 80 2954 9789 2354 6057 5977 5345 9970 8453 9037 8728 8280 9449 5789 3546 4900 2865 8314 1585 1854 8320 7556 8782 4151 5635 8235 547 9633 4731 6431 3117 5891 9385 2905 8245 6520 9961 3589 6489 8413 3704 6295 7771 3152 2083 1317 9131 6028 709 1794 7882 9029 9350 7742 4258 6063 7055 5885 5695 1786 2315 8812 7677 2778 2795 5921 9298 3834 589 6866 3326 4293 4239 1096 8525 7402 3492 7655 3429 4201 528 2389 4309 9878 130 9646 7020 7185 5530 2714 49 8924 2605 7726 2780 5400 4726 2078 9235 5315 22 2560 686 4261 4735 289 2741 9307 9024 6170 4586 9552 9638 9974 9429 846 9620 6448 8032 6228 241 8081 5152 2846 6886 7932 9325 1611 1088 9638 8005 2189 3276 9770 7530 8012 59 270 8397 9083 7520 2982 8634 7157 2956 9141 8004 3654 6668 7114 9882 6909 6273 6112 833 3158 4044 1236 5848 6211 874 3852 8401 4150 3622 5930 3240 4760 6200 2715 38

In [ ]:

%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda.h>

#define SIZE 1024
#define THREADS_PER_BLOCK 256

using namespace std;

__global__ void bubbleSort(int *arr, int n) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int start = 2 * index * THREADS_PER_BLOCK;
    for(int i = 0; i < n; i++) {
        for(int j = start; j < min(start+2*THREADS_PER_BLOCK, n-1); j++) {
            if (arr[j] > arr[j+1]) {
                int temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
    }
}

void imprimirArr(int arr[], int n) {
    for(int i = 0; i < n; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
}

int main() {
    srand(time(0));
    int arr[SIZE];
    int *d_arr;
    size_t size = SIZE * sizeof(int);

    for(int i = 0; i < SIZE; i++) {
        arr[i] = rand() % 10001;
    }

    cout << "Arreglo desordenado: " << endl;
    imprimirArr(arr, SIZE);


    cudaMalloc((void **)&d_arr, size);

    cudaMemcpy(d_arr, arr, size, cudaMemcpyHostToDevice);

    bubbleSort<<<SIZE/THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(d_arr, SIZE);

    cudaMemcpy(arr, d_arr, size, cudaMemcpyDeviceToHost);

    cout << "Arreglo ordenado: " << endl;
    imprimirArr(arr, SIZE);

    cudaFree(d_arr);

    return 0;
}



Arreglo desordenado: 
8999 2179 4053 830 6069 8749 7484 485 9697 5118 8156 4300 218 965 2422 9355 8207 5325 865 6298 9943 7769 1286 6662 5926 7221 2240 8047 7726 327 6770 6724 3586 1901 7554 9655 1728 6116 1218 1425 2313 9374 6804 3610 1418 9227 2964 9625 4551 3829 7001 4493 1597 8287 1154 8602 6587 4473 7728 4312 5880 4497 2114 9466 6398 746 198 9205 6862 1416 629 9175 789 8513 2784 3286 7739 6827 2910 2289 655 989 7861 3331 355 93 3012 6942 4567 1818 2332 446 6315 4446 990 3791 5192 1188 2995 3132 3683 4704 3386 5551 3216 7249 8838 2033 4076 2826 4322 5810 3816 2182 220 4171 3354 4311 2191 7921 6129 5602 9446 3522 47 435 7313 6318 2702 1386 528 6385 7169 3914 1936 1463 1163 773 3496 5239 3599 8897 2127 8494 2157 2347 2664 5512 6658 5934 4511 2786 1535 3957 6308 2661 4392 4699 57 8174 7165 586 4558 4333 4500 6494 6876 6742 8346 371 3059 3024 9269 5187 1517 2504 8613 5261 8016 5271 1194 2527 9136 2730 6484 6523 6470 1954 1221 6528 127 9465 7114 5765 4877 2692 3337 1752 513 1683 3202 35

In [ ]:
%%cu
#include <iostream>
#include <cstdlib>
#include <ctime>
#include <cuda.h>

#define SIZE 1024
#define THREADS_PER_BLOCK 256

using namespace std;

__global__ void bubbleSort(int *arr, int n) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int start = 2 * index * THREADS_PER_BLOCK;
    for(int i = 0; i < n; i++) {
        for(int j = start; j < min(start+2*THREADS_PER_BLOCK, n-1); j++) {
            if (arr[j] > arr[j+1]) {
                int temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
        __syncthreads(); // Sincronización de hilos después de cada iteración
    }
}

void imprimirArr(int arr[], int n) {
    for(int i = 0; i < n; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
}

int main() {
    srand(time(0));
    int arr[SIZE];
    int *d_arr;
    size_t size = SIZE * sizeof(int);

    for(int i = 0; i < SIZE; i++) {
        arr[i] = rand() % 10001;
    }

    cout << "Arreglo desordenado: " << endl;
    imprimirArr(arr, SIZE);

    cudaMalloc((void **)&d_arr, size);

    cudaMemcpy(d_arr, arr, size, cudaMemcpyHostToDevice);

    bubbleSort<<<SIZE/THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(d_arr, SIZE);

    cudaMemcpy(arr, d_arr, size, cudaMemcpyDeviceToHost);

    cout << "Arreglo ordenado: " << endl;
    imprimirArr(arr, SIZE);

    cudaFree(d_arr);

    return 0;
}


Arreglo desordenado: 
3675 4844 2569 7479 2787 6999 7813 140 8586 1757 159 674 2416 6930 2713 6965 7794 8547 277 5529 9450 3936 9796 2575 6090 1128 625 8883 7290 6482 9137 964 1325 2785 9523 4113 9784 7335 5332 8370 9093 5491 9044 2587 2420 2835 630 213 2460 907 5743 2988 5923 6617 6643 2012 8824 7268 1973 6113 3749 2189 8157 5075 4974 7679 266 4757 5013 5598 4205 5184 1088 4328 8850 4587 8242 9480 4801 1781 1466 1622 4769 7389 9318 1411 9401 8141 8679 2453 4253 2428 4642 3488 8582 694 1166 8848 6530 7259 4445 735 3521 6612 5063 2371 1198 4383 2929 7078 6164 4395 9779 933 1783 9096 2344 2263 7236 2102 4716 2568 5609 436 7135 4190 1130 9381 4116 7660 6639 8561 9474 1238 6251 5615 3609 7449 9999 6539 5606 6162 2012 5384 7095 3796 4480 518 6059 2794 2620 1853 6441 9308 2289 3576 3497 4498 2956 7613 3236 673 7252 2710 1911 3502 8325 6600 2029 9402 4217 7635 5564 6229 4098 3737 1103 9657 4255 8241 3529 7954 93 9971 7261 3461 4625 1836 9038 7581 9449 2274 9333 7779 6063 2322 2359 4387 0 4389